<h1>Segmentation and Clustering Toronto Neighborhoods</h1>

Import required modules

In [105]:
!conda install -c conda-forge wikipedia --yes
import pandas as pd
import numpy as np
import wikipedia as wp #For web scraping wikipedia pages

import io
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [61]:
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans

Let's web scrape Wikipedia Canada's Toronto Neighborhoods ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

And convert it into pandas DataFrame

In [37]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode('UTF-8')
df = pd.read_html(html, header=0)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove 'Not Assigned' Borough rows

In [38]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

In [43]:
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Let's check for any null values

In [40]:
df.isnull().sum()

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64

Let's check the shape of the Data frame

In [41]:
df.shape

(103, 3)

Get the geographical locations from Geospatial_data and fetch Toronto Neighborhoods location details by Postal Code

In [48]:
url = "https://cocl.us/Geospatial_data"
lat_long = requests.get(url).text
lat_long_df=pd.read_csv(io.StringIO(lat_long))
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge both dataframes (df & lat_long_df) on column 'Postal Code'

In [49]:
toronto_df = pd.merge(df, lat_long_df, on='Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [50]:
toronto_df.shape

(103, 5)

In [52]:
toronto_df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

# Data preprocessing

Let's explore and segment 'North York' Borough venues

In [65]:
nyork_df = toronto_df[toronto_df['Borough'] == 'North York'].reset_index(drop=True)
nyork_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


Fetch North York Borough's location details

In [102]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent = 'ny_explorer')
nyork_location = geolocator.geocode(address)
nyork_latitude = nyork_location.latitude
nyork_longitude = nyork_location.longitude
print(f'North York, Toronto location [{nyork_latitude},{nyork_longitude}]')

North York, Toronto location [43.7543263,-79.44911696639593]


Visualize 'North York' Borough neighborhoods using Folium

In [103]:
def create_folium_map(df, latitude, longitude):
    nyork_map = folium.Map(location = [latitude, longitude], zoom_start=10)

    for lat, lon, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
        label = '{}: {}'.format(borough, neighborhood)
        label = folium.Popup(label, parse_html = True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup = label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(nyork_map) 

    return(nyork_map)
           

In [104]:
nyork_map = create_folium_map(nyork_df, nyork_latitude, nyork_longitude)
nyork_map

Now using Foursquare let's explore nearby venues within 500 meters for every neighborhood of 'North York'

In [75]:
CLIENT_ID = '42AR4F3AMXTVFCE1H44M13RPLB0QG3GO2BYQ4PS4V10YB11Z' # your Foursquare ID
CLIENT_SECRET = 'GLLQBZRQO20UUC4WNCDRI2FSS5IFGFMJRDCQHFIY3PBA3SDE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 42AR4F3AMXTVFCE1H44M13RPLB0QG3GO2BYQ4PS4V10YB11Z
CLIENT_SECRET:GLLQBZRQO20UUC4WNCDRI2FSS5IFGFMJRDCQHFIY3PBA3SDE


Let's explore the sample Foursquare API response format for the first Neighborhood of 'North York'

In [71]:
nyork_df.loc[0, 'Neighborhood']

'Parkwoods'

Fetch 'Parkwoods' location details using geopy

In [73]:
neighborhood_lat = nyork_df.loc[0, 'Latitude']
neighborhood_lng = nyork_df.loc[0, 'Longitude']
neighborhood_name = nyork_df.loc[0, 'Neighborhood']
neighborhood_borough = nyork_df.loc[0, 'Borough']
print(f'{neighborhood_borough}, {neighborhood_name}, {neighborhood_lat}, {neighborhood_lng}')

North York, Parkwoods, 43.7532586, -79.3296565


In [78]:
LIMIT = 100
radius = 2000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_lat, neighborhood_lng, VERSION, radius, LIMIT)

print(url)


https://api.foursquare.com/v2/venues/explore?client_id=42AR4F3AMXTVFCE1H44M13RPLB0QG3GO2BYQ4PS4V10YB11Z&client_secret=GLLQBZRQO20UUC4WNCDRI2FSS5IFGFMJRDCQHFIY3PBA3SDE&ll=43.7532586,-79.3296565&v=20180605&radius=2000&limit=100


In [85]:
results = requests.get(url).json()
# results

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [81]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'][0]
# nearby_venues.head()
# # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Donalda Golf & Country Club,Golf Course,43.752816,-79.342741
2,Brookbanks Park,Park,43.751976,-79.332140
3,Island Foods,Caribbean Restaurant,43.745866,-79.346035
4,Galleria Supermarket,Supermarket,43.753520,-79.349518


In [82]:
nearby_venues.shape

(88, 4)

Now let's repeat this process for all neighborhoods of North York

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
nyork_venues = getNearbyVenues(names=nyork_df['Neighborhood'],
                                   latitudes=nyork_df['Latitude'],
                                   longitudes=nyork_df['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [86]:
nyork_venues.shape

(245, 7)

No.of unique catgorie venues of North York Borough

In [87]:
print('There are {} uniques categories.'.format(len(nyork_venues['Venue Category'].unique())))

There are 106 uniques categories.


Dealing with categorical variables (Category Venues)

In [88]:
# one hot encoding
nyork_dummies = pd.get_dummies(nyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyork_dummies['Neighborhood'] = nyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyork_dummies.columns[-1]] + list(nyork_dummies.columns[:-1])
nyork_dummies = nyork_dummies[fixed_columns]

nyork_dummies.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
nyork_grouped = nyork_dummies.groupby('Neighborhood').mean().reset_index()
nyork_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.095238,0.0,...,0.0,0.047619,0.0,0.000000,0.0,0.000000,0.0,0.047619,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.041667,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.041667,0.0,0.041667,0.0,0.041667,0.0,0.000000,0.0,0.0
3,Don Mills,0.0,0.000000,0.000000,0.0,0.074074,0.037037,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,Downsview,0.0,0.052632,0.000000,0.0,0.000000,0.052632,0.0,0.052632,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0


Let's print top 5 common values of each neighborhood

In [91]:
num_top_venues = 5

for hood in nyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyork_grouped[nyork_grouped['Neighborhood'] == hood].T.reset_index()    
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                Supermarket  0.05
3  Middle Eastern Restaurant  0.05
4                       Park  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3         Pizza Place  0.04
4                Café  0.04


----Don Mills----
                 venue  freq
0                 Café  0.07
1           Restaurant  0.07
2     Asian Restaurant  0.07
3                  Gym  0.07
4  Japanese Restaurant  0.07


----Downsview----
              venue  freq
0     Grocery Store  0.16
1              Park  0.11
2  Business Service  0.11
3   

Now let's create a dataframe with top 5 venues

In [93]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyork_grouped['Neighborhood']

for ind in np.arange(nyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Restaurant,Chinese Restaurant
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Food Court
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Breakfast Spot
3,Don Mills,Café,Gym,Japanese Restaurant,Restaurant,Beer Store
4,Downsview,Grocery Store,Business Service,Park,Hotel,Bus Stop


# Unsupervised Machine Learning Clustering

Implement K-Means algorithm

In [111]:
# set number of clusters
kclusters = 4

nyork_grouped_clustering = nyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 2])

Create new column and add cluster labels

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyork_merged = nyork_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyork_merged = nyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyork_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Women's Store,Dim Sum Restaurant,Clothing Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Financial or Legal Service,Hockey Arena,Portuguese Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop
3,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Café,Gym,Japanese Restaurant,Restaurant,Beer Store
4,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Japanese Restaurant,Sushi Restaurant,Pub,Clothing Store


In [117]:
nyork_merged.drop(['Postal Code'], axis=1, inplace=True)
nyork_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Women's Store,Dim Sum Restaurant,Clothing Store
1,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Financial or Legal Service,Hockey Arena,Portuguese Restaurant
2,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop
3,North York,Don Mills,43.745906,-79.352188,0.0,Café,Gym,Japanese Restaurant,Restaurant,Beer Store
4,North York,Glencairn,43.709577,-79.445073,0.0,Park,Japanese Restaurant,Sushi Restaurant,Pub,Clothing Store


In [148]:
nyork_merged['Cluster Labels'] = nyork_merged['Cluster Labels'].astype('int')

In [149]:
nyork_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,1,Food & Drink Shop,Park,Women's Store,Dim Sum Restaurant,Clothing Store
1,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Financial or Legal Service,Hockey Arena,Portuguese Restaurant
2,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop
3,North York,Don Mills,43.745906,-79.352188,0,Café,Gym,Japanese Restaurant,Restaurant,Beer Store
4,North York,Glencairn,43.709577,-79.445073,0,Park,Japanese Restaurant,Sushi Restaurant,Pub,Clothing Store


We've reached to end step. Now let's visualize the clusters

In [150]:
# create map
map_clusters = folium.Map(location=[nyork_latitude, nyork_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyork_merged['Latitude'], nyork_merged['Longitude'], nyork_merged['Neighborhood'], nyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters